In [ ]:
import os
import math
import requests
import pandas as pd
import geopandas as gpd
import osmnx as ox
import pyrosm
import rasterio
import rasterio.plot
from rasterio import features
from rasterio.warp import Resampling
from rasterio.transform import Affine
import numpy as np
from shapely.geometry import box, Point
from pathlib import Path
import matplotlib.pyplot as plt
from rasterio.warp import Resampling


# Magic command to ensure plots render correctly in the notebook
%matplotlib inline


print("All packages imported successfully.")

In [ ]:
# --- Constants ---
TARGET_CRS = "EPSG:32632"  # UTM Zone 32N
BUFFER_METERS = 1000       # 1km buffer
PIXEL_SIZE = 10            # 10m resolution
AOI_NAME = "Kanton Schaffhausen" # Geocoding query

# --- Directories ---
BASE_DIR = Path.cwd().parent.parent
DATA_DIR = BASE_DIR / "data"
RESULTS_DIR = BASE_DIR / "results"
TEMP_DIR = RESULTS_DIR / "temp_files"

# Create directories if they don't exist
DATA_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)
TEMP_DIR.mkdir(exist_ok=True)

# --- Input File Paths ---
# Cost tables
OSM_COST_CSV_PATH = DATA_DIR / "osm_resistance_costs.csv"
CLC_COST_CSV_PATH = DATA_DIR / "clc_resistance_costs.csv"

# Raw data (will be downloaded)
PBF_PATH_DE = DATA_DIR / "baden-wuerttemberg-latest.osm.pbf"
PBF_PATH_CH = DATA_DIR / "switzerland-latest.osm.pbf"

# --- Processed Cache File Paths ---
PROCESSED_OSM_CACHE = DATA_DIR / "processed_osm_data.gpkg"
CLC_RECLASSIFIED_PATH = TEMP_DIR / "temp_raster_clc_reclassified.tif"
CLC_PRECLIPPED_VECTOR_PATH = DATA_DIR / "U2018_CLC2018_V2020_20u1.gpkg" 

# --- Final Output ---
FINAL_RASTER = RESULTS_DIR / "final_resistance_surface.tif"

print(f"Base Directory:    {BASE_DIR}")
print(f"Data Directory:    {DATA_DIR}")
print(f"Results Directory: {RESULTS_DIR}")
print(f"Temp Directory:    {TEMP_DIR}")

In [ ]:
# --- AOI Definition ---
print(f"1. Defining AOI from '{AOI_NAME}'...")

# Get Schaffhausen boundary in WGS84 via OSMnx geocoding
gdf_sh_wgs = ox.geocode_to_gdf(AOI_NAME)

# Reproject to target CRS (UTM 32N)
gdf_sh_proj = gdf_sh_wgs.to_crs(TARGET_CRS)

# Buffer the polygon first
print(f"   Buffering polygon by {BUFFER_METERS}m...")
buffered_polygon = gdf_sh_proj.buffer(BUFFER_METERS)
bounds = buffered_polygon.total_bounds

# Create a Shapely polygon for the final AOI
aoi_poly = box(*bounds)
print(f"   Final rectangular AOI defined.")

# Create a WGS84 version of the AOI for data fetching
aoi_poly_wgs = gpd.GeoSeries([aoi_poly], crs=TARGET_CRS).to_crs("EPSG:4326").iloc[0]
aoi_bounds_wgs = aoi_poly_wgs.bounds

aoi_gdf = gpd.GeoDataFrame(geometry=[aoi_poly], crs=TARGET_CRS)

# Plot the AOI vs canton boundary
fig, ax = plt.subplots(figsize=(8, 8))
gdf_sh_proj.boundary.plot(ax=ax, edgecolor='blue', label='Canton Boundary')
aoi_gdf.boundary.plot(ax=ax, edgecolor='red', linestyle='--', label='AOI Boundary')
ax.set_title('AOI vs Canton Schaffhausen Boundary')
ax.legend()
plt.show()

In [ ]:
# --- Master Grid Definition ---
print("2. Defining Master Grid from AOI...")

# Get the bounds from AOI polygon
min_x, min_y, max_x, max_y = aoi_poly.bounds

# Calculate the grid dimensions (shape)
width = math.ceil((max_x - min_x) / PIXEL_SIZE)
height = math.ceil((max_y - min_y) / PIXEL_SIZE)

# Create the Rasterio 'transform'
transform = Affine.translation(min_x, max_y) * Affine.scale(PIXEL_SIZE, -PIXEL_SIZE)

# Store the grid metadata
master_grid_meta = {
    'crs': TARGET_CRS,
    'transform': transform,
    'height': height,
    'width': width,
    'driver': 'GTiff'
}
master_shape = (height, width)

print(f"   Master Grid defined:")
print(f"   Shape (h, w): {master_shape}")
print(f"   Resolution: {PIXEL_SIZE}m")

In [ ]:
# --- Download Raw OSM .pbf Files ---
print("3. Checking for required .pbf files...")

# Define file paths and download URLs
PBF_URL_DE = "https://download.geofabrik.de/europe/germany/baden-wuerttemberg-latest.osm.pbf"
PBF_URL_CH = "https://download.geofabrik.de/europe/switzerland-latest.osm.pbf"
files_to_download = [(PBF_PATH_DE, PBF_URL_DE), (PBF_PATH_CH, PBF_URL_CH)]

# --- Download function ---
def download_file(url, local_path):
    print(f"   Downloading {local_path.name}... (This may take 2-10 minutes)")
    try:
        with requests.get(url, stream=True, timeout=30) as r:
            r.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): 
                    f.write(chunk)
        print(f"   Download complete: {local_path.name}")
    except Exception as e:
        print(f"!!! ERROR downloading {url}: {e}")
        if local_path.exists(): os.remove(local_path)
        raise

# --- Loop and check ---
for path, url in files_to_download:
    if not path.exists():
        print(f"   '{path.name}' not found.")
        download_file(url, path)
    else:
        print(f"   Found '{path.name}'.")

print("   All required .pbf files are present.")

In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np
import gc

# --- 4. Process Corine ---

print(f"4. Checking for processed Corine file ('{CLC_RECLASSIFIED_PATH.name}')...")

if CLC_RECLASSIFIED_PATH.exists():
    print(f"   Found '{CLC_RECLASSIFIED_PATH.name}'. Skipping processing.")
else:
    print(f"   Processing Corine data (Simple Vector Workflow)...")

    DEFAULT_CLC_RESISTANCE = 10000.0 # High-cost/barrier

    if not CLC_PRECLIPPED_VECTOR_PATH.exists():
        print(f"!!! ERROR: Pre-clipped CLC Vector file not found at '{CLC_PRECLIPPED_VECTOR_PATH}'")
        raise FileNotFoundError(f"Missing pre-clipped CLC file: {CLC_PRECLIPPED_VECTOR_PATH}")

    try:
        # 1. Load the Resistance map (Lookup 1)
        clc_res_map_df = pd.read_csv(CLC_COST_CSV_PATH)
        clc_res_map_df['clc_code'] = clc_res_map_df['clc_code'].astype(int)
        print(f"   Loaded {len(clc_res_map_df)} resistance rules.")

        # 2. Load your pre-clipped CLC Vector Data
        print(f"   Loading pre-clipped CLC vector: '{CLC_PRECLIPPED_VECTOR_PATH.name}'...")
        clc_gdf = gpd.read_file(CLC_PRECLIPPED_VECTOR_PATH)
        print(f"   ...Loaded {len(clc_gdf)} polygons.")

        # 3. Reproject the CLC data to the Master Project CRS
        clc_proj = clc_gdf.to_crs(master_grid_meta['crs'])
        
        del clc_gdf
        gc.collect()

        # 4. Clip the reprojected data to the precise AOI polygon
        print("   Clipping reprojected CLC to precise AOI geometry...")
        clc_clipped = gpd.clip(clc_proj, aoi_gdf.geometry)
        print(f"   ...Clipping complete. {len(clc_clipped)} polygons remaining.")
        
        del clc_proj
        gc.collect()
        
        if clc_clipped.empty:
            print("!!! ERROR: Clipping resulted in an empty GeoDataFrame. Check AOI CRS.")
            raise ValueError("CLC clipping failed.")

        # --- 5. Merge Resistance Values ---
        print("   Merging resistance values...")
        clc_clipped['Code_18'] = clc_clipped['Code_18'].astype(int)

        clc_with_costs = clc_clipped.merge(
            clc_res_map_df,
            left_on='Code_18',
            right_on='clc_code'
        )
        
        if clc_with_costs.empty:
            print("!!! ERROR: Merging CLC with resistance CSV resulted in no matches.")
            print("!!! Check 'Code_18' column in your vector file and 'clc_code' in your CSV.")
            raise ValueError("CLC resistance merge failed.")

        # Save Intermediate Vector
        temp_vector_path = TEMP_DIR / "temp_vector_clc_reclassified.gpkg"
        
        # Save only essential columns for inspection
        clc_with_costs[['Code_18', 'clc_code', 'resistance', 'geometry']].to_file(temp_vector_path, driver="GPKG")
        print(f"   ...Intermediate vector saved to '{temp_vector_path.name}'")

        # 6. Rasterize the vector data
        print("   Rasterizing vector polygons...")
        
        shapes = [(row.geometry, row.resistance) 
                  for row in clc_with_costs.itertuples() 
                  if row.geometry is not None and not row.geometry.is_empty]

        clc_reclassified_data = np.full(master_shape, DEFAULT_CLC_RESISTANCE, dtype=np.float32)

        features.rasterize(
            shapes=shapes,
            out=clc_reclassified_data,
            transform=master_grid_meta['transform'],
            all_touched=True,
            fill=DEFAULT_CLC_RESISTANCE,
            dtype=np.float32
        )
        
        # 7. Save the final reclassified raster
        # (This path remains unchanged as it is a FINAL output, not temporary)
        meta_to_save = master_grid_meta.copy()
        meta_to_save.update(dtype=np.float32, count=1, nodata=None)
        
        with rasterio.open(CLC_RECLASSIFIED_PATH, 'w', **meta_to_save) as dst:
            dst.write(clc_reclassified_data, 1)
        print(f"   ...Corine fallback raster saved to '{CLC_RECLASSIFIED_PATH.name}'")

    except Exception as e:
        print(f"!!! Error processing Corine vector data: {e} !!!")
        raise

In [ ]:
# --- Process OSM Data ---
print(f"5. Checking for processed OSM cache ('{PROCESSED_OSM_CACHE.name}')...")

# Check if a pre-processed version of the OSM data exists (e.g., from a previous run).
# This is a critical efficiency check to save hours of processing time.
if PROCESSED_OSM_CACHE.exists():
    print(f"   Found fast-loading cache file. Loading...")
    # Load the cached GeoPackage (GPKG) file directly into a GeoDataFrame
    osm_gdf = gpd.read_file(PROCESSED_OSM_CACHE)
    print(f"   ...Cache loaded. Fetched {len(osm_gdf)} features.")
else:
    print(f"   Cache file not found. Starting full data processing...")
    
    # 1. Prepare Tags and Bounding Box
    # Load the resistance cost rules
    resistance_df = pd.read_csv(OSM_COST_CSV_PATH)
    # Extract all unique 'osm_key' tags (e.g., 'landuse', 'highway') from your CSV rules.
    needed_keys = resistance_df['osm_key'].unique().tolist()
    # Create a pyrosm-compatible filter dictionary {key: True} to fetch *all* values for those keys.
    tags_filter = {key: True for key in needed_keys}
    # Convert the WGS84 bounding box tuple (minx, miny, maxx, maxy) to a list for pyrosm.
    bbox_wgs_list = [aoi_bounds_wgs[0], aoi_bounds_wgs[1], aoi_bounds_wgs[2], aoi_bounds_wgs[3]]

    # 2. Extract Data for Germany (Baden-Württemberg)
    print("   Processing German data (Baden-Württemberg)... THIS IS SLOW.")
    # Initialize pyrosm object with the German PBF file, clipped to the AOI box.
    osm_de = pyrosm.OSM(str(PBF_PATH_DE), bounding_box=bbox_wgs_list) 
    # Extract all features matching the defined tags (e.g., all roads, all forests).
    osm_gdf_de = osm_de.get_data_by_custom_criteria(custom_filter=tags_filter)
    print("   ...German data complete.")

    # 3. Extract Data for Switzerland (Schaffhausen area)
    print("   Processing Swiss data... THIS IS SLOW.")
    # Initialize pyrosm object with the Swiss PBF file, clipped to the AOI box.
    osm_ch = pyrosm.OSM(str(PBF_PATH_CH), bounding_box=bbox_wgs_list)
    # Extract all features matching the defined tags.
    osm_gdf_ch = osm_ch.get_data_by_custom_criteria(custom_filter=tags_filter)
    print("   ...Swiss data complete.")

    # 4. Combine and Clean
    print("   Combining and cleaning data...")
    # Concatenate the GeoDataFrames from Germany and Switzerland.
    # .drop_duplicates(subset=['id']) removes any features duplicated near the border.
    osm_gdf_wgs = pd.concat([osm_gdf_de, osm_gdf_ch]).drop_duplicates(subset=['id'])

    # Delete large objects and force garbage collection to free memory after processing PBFs.
    del osm_gdf_de, osm_gdf_ch, osm_de, osm_ch
    gc.collect()

    # Filter columns to keep only 'id', 'geometry', and the specific tags required for resistance assignment.
    needed_cols = ['id', 'geometry'] + needed_keys
    cols_to_keep = [col for col in needed_cols if col in osm_gdf_wgs.columns]
    osm_gdf_wgs = osm_gdf_wgs[cols_to_keep]

    # 5. Reproject to Master CRS
    print("   Reprojecting OSM data...")
    # Convert the data from WGS84 (latitude/longitude, the OSM standard) to the project's CRS (EPSG:32632).
    # This is essential for distance calculations and aligning with the master grid.
    osm_gdf = osm_gdf_wgs.to_crs(TARGET_CRS)

    # 6. Final Data Integrity Checks
    # Remove rows where geometry is missing (notna()).
    osm_gdf = osm_gdf[osm_gdf.geometry.notna()]
    # Filter to keep only features that are usable in the resistance model (Polygons for landcover, LineStrings for roads/rivers).
    osm_gdf = osm_gdf[osm_gdf.geometry.geom_type.isin(['Polygon', 'LineString', 'MultiPolygon', 'MultiLineString'])]
    print(f"   Fetched and processed {len(osm_gdf)} relevant OSM features.")
    
    # 7. Cache the Result
    print(f"   Saving processed data to cache file: '{PROCESSED_OSM_CACHE.name}'")
    # Save the cleaned, reprojected data as a GeoPackage for fast loading next time.
    osm_gdf.to_file(PROCESSED_OSM_CACHE, driver="GPKG")
    print("   ...Cache file saved.")

natural water makes problems

In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
import numpy as np
import os

# --- 1. Setup Paths and Metadata ---
print("6. Setting up for intermediate file generation...")

# Define and create the new temp directory
TEMP_DIR = os.path.join(RESULTS_DIR, "temp_files")
os.makedirs(TEMP_DIR, exist_ok=True)
print(f"   ...Intermediate files will be saved in: {TEMP_DIR}")

# Load the resistance rules from our new CSV
resistance_df = pd.read_csv(OSM_COST_CSV_PATH)

# Metadata for all new intermediate rasters (based on master grid)
meta_to_save = master_grid_meta.copy()
meta_to_save.update(dtype=np.float32, count=1, nodata=0.0) # Use 0.0 as NoData

# --- 2. Generate Intermediate Files for each OSM Key ---
print("7. Generating intermediate vector and raster files for each osm_key...")

# Get all unique keys from the CSV
all_osm_keys = resistance_df['osm_key'].unique()

for osm_key in all_osm_keys:
    print(f"\nProcessing key: '{osm_key}'")
    
    # Get all rules from the CSV for this specific key
    key_rules = resistance_df[resistance_df['osm_key'] == osm_key].sort_values('priority', ascending=True)
    
    if key_rules.empty:
        print(f"  ...No rules found for '{osm_key}' in CSV. Skipping.")
        continue
        
    # Check if this key even exists as a column in the main GeoDataFrame
    if osm_key not in osm_gdf.columns:
        print(f"  ...Column '{osm_key}' not in GeoDataFrame. Skipping.")
        continue

    # --- 2a. Create Intermediate Vector File (.gpkg) ---
    
    # Get all unique values we need to filter for this key
    relevant_values = key_rules['osm_value'].unique()
    
    if 'yes' in relevant_values:
        # Handle tags like 'building=yes' which means any non-null value
        features_for_key_gdf = osm_gdf[osm_gdf[osm_key].notna()].copy()
    else:
        # Filter for all other specific values (e.g., 'highway' in ['path', 'residential', ...])
        features_for_key_gdf = osm_gdf[osm_gdf[osm_key].isin(relevant_values)].copy()

    if features_for_key_gdf.empty:
        print(f"  ...No geometries found in GDF for key '{osm_key}'. Skipping.")
        continue
        
    # Add the resistance value to the intermediate GDF for inspection in QGIS
    # Create a mapping dictionary {value: resistance}
    value_resistance_map = key_rules.set_index('osm_value')['resistance'].to_dict()
    
    if 'yes' in relevant_values:
         # For 'yes', apply the resistance to *all* non-null values
         resistance_val = value_resistance_map.get('yes', 0) # Get resistance for 'yes'
         features_for_key_gdf['resistance'] = resistance_val
    else:
         features_for_key_gdf['resistance'] = features_for_key_gdf[osm_key].map(value_resistance_map)

    # Save the intermediate vector file
    temp_vector_path = os.path.join(TEMP_DIR, f"temp_vector_{osm_key}.gpkg")
    try:
        # We only need a few columns for inspection
        cols_to_save = ['id', osm_key, 'geometry', 'resistance']
        features_for_key_gdf[cols_to_save].to_file(temp_vector_path, driver="GPKG")
        print(f"  ...Saved intermediate vector: {os.path.basename(temp_vector_path)}")
    except Exception as e:
        print(f"  ...ERROR saving vector {temp_vector_path}: {e}")

    # --- 2b. Create Intermediate Raster File (.tif) ---
    
    # Initialize an empty raster array for this key
    key_raster = np.full(master_shape, 0.0, dtype=np.float32)
    temp_raster_path = os.path.join(TEMP_DIR, f"temp_raster_{osm_key}.tif")

    # Now, iterate through the rules for this key (which are sorted by priority)
    # This will "burn" features on top of each other *correctly*
    # (e.g., 'highway=motorway' will burn over 'highway=path')
    
    print(f"  ...Rasterizing {len(key_rules)} rules for '{osm_key}' by priority...")
    for _, row in key_rules.iterrows():
        key, value, resistance = row['osm_key'], row['osm_value'], row['resistance']
        
        # We can re-use the GDF we filtered earlier
        if value == 'yes':
            features_to_rasterize = features_for_key_gdf # Already filtered for notna()
        else:
            features_to_rasterize = features_for_key_gdf[features_for_key_gdf[key] == value]
        
        if features_to_rasterize.empty:
            continue

        # Create (geom, value) pairs for rasterio
        shapes = [(geom, resistance) for geom in features_to_rasterize.geometry]
        
        # Rasterize (burn) these features into the key_raster
        features.rasterize(
            shapes=shapes,
            out=key_raster,
            transform=master_grid_meta['transform'],
            merge_alg=MergeAlg.replace,
            all_touched=True, # Essential for linear features (roads, rivers, barriers)
            dtype=np.float32
        )
    
    # Save the final intermediate raster for this key
    with rasterio.open(temp_raster_path, 'w', **meta_to_save) as dst:
        dst.write(key_raster, 1)
    print(f"  ...Saved intermediate raster: {os.path.basename(temp_raster_path)}")

print("\nIntermediate file generation complete.")

In [ ]:
# --- 7. Combine All Rasters ---
print("7. Combining all intermediate rasters into final surface...")

try:
    # --- 1. Load the Base CLC Fallback Raster ---
    print(f"   ...Loading CLC fallback raster: {CLC_RECLASSIFIED_PATH.name}")
    with rasterio.open(CLC_RECLASSIFIED_PATH) as src:
        # This is the base map.
        final_raster = src.read(1).astype(np.float32)
        meta = src.meta.copy()
        meta.update(dtype=np.float32, nodata=None) # Final raster has no nodata
    
    # --- 2. Define Intermediate File Location ---
    TEMP_DIR = os.path.join(RESULTS_DIR, "temp_files")
    
    # Find all the intermediate OSM rasters we just created
    # We use glob to get a list of all matching files
    osm_raster_files = list(Path(TEMP_DIR).glob("temp_raster_*.tif"))
    
    if not osm_raster_files:
        print(f"!!! ERROR: No 'temp_raster_*.tif' files found in {TEMP_DIR}")
        print("!!! Please re-run the previous cell (Generate Intermediate Rasters) !!!")
    
    print(f"   ...Found {len(osm_raster_files)} intermediate OSM rasters to combine.")

    # --- 3. Loop, Load, and Combine ---
    for raster_file in osm_raster_files:
        print(f"   ...Merging: {raster_file.name}")
        with rasterio.open(raster_file) as src:
            # Read the OSM layer
            osm_layer = src.read(1)
            
            # Clean the layer: our nodata value was 0.0
            # np.nan_to_num is a fast way to replace any potential NaNs with 0.0
            osm_layer = np.nan_to_num(osm_layer, nan=0.0)
            
            # Use np.maximum to combine.
            # This burns barriers (high values) over landcover (low values)
            # AND fills gaps (where osm_layer is 0) with CLC values.
            final_raster = np.maximum(final_raster, osm_layer)

    # --- 4. Save the Final Combined Raster ---
    with rasterio.open(FINAL_RASTER, 'w', **meta) as dst:
        dst.write(final_raster, 1)
        
    print(f"\n   SUCCESS: Final resistance surface saved to '{FINAL_RASTER}'")

except FileNotFoundError as e:
    print(f"!!! ERROR: Could not find base CLC file: {e}")
    print("!!! Please re-run the CLC processing cell. !!!")
except Exception as e:
    print(f"!!! An unexpected error occurred during combination: {e}")
    raise

In [ ]:
# --- Visualize Final Result ---
print("Plotting final cost surface...")

try:
    with rasterio.open(FINAL_RASTER) as src:
        cost_data = src.read(1)

        print(f'   Min value: {np.min(cost_data):.2f}, Max value: {np.max(cost_data):.2f}')

        # Print amount of nan values
        nan_count = np.isnan(cost_data).sum()
        print(f'   Number of NaN values in final raster: {nan_count}')

        print(min(cost_data.flatten()), max(cost_data.flatten()))
        
        # Use a log scale for visualization
        cost_data_log = np.log1p(cost_data)
        
        fig, ax = plt.subplots(figsize=(12, 12))
        
        im = rasterio.plot.show(
            cost_data_log,
            transform=src.transform,
            ax=ax,
            cmap='RdYlGn_r'
        )
        
        cbar = fig.colorbar(im.images[0], ax=ax, shrink=0.7)
        cbar.set_label('Log(Resistance Cost)')
        
        # Plot the canton boundary on top
        gdf_sh_proj.boundary.plot(ax=ax, color='blue', linestyle='--', linewidth=2, label='Canton Boundary')
        
        ax.legend()
        ax.set_title("Final Cost Surface (Log Scale)")
        ax.set_xlabel("Easting (m)")
        ax.set_ylabel("Northing (m)")
        
        plt.tight_layout()
        plt.show()

except NameError:
    print("!!! ERROR: 'gdf_sh_proj' not found. Please re-run Cell 4 (AOI Definition).")
except FileNotFoundError:
    print(f"!!! ERROR: File not found at '{FINAL_RASTER}'")
    print("Please make sure Cell 10 (Combine Final Raster) ran successfully.")

In [ ]:
# plot water courses and water bodies from the clc fallback raster
import rasterio
from rasterio.plot import show as rioshow
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

# --- ASSUMPTIONS (Ensure these variables are defined) ---
# CLC_RECLASSIFIED_PATH (Path object to your CLC reclassified raster)
# aoi_gdf (GeoDataFrame of the AOI boundary)

# Define the target resistance value for water features (511 and 512 in our cost table)
WATER_RESISTANCE_VALUE = 5000

print(f"Loading CLC fallback raster from {CLC_RECLASSIFIED_PATH.name}")

with rasterio.open(CLC_RECLASSIFIED_PATH) as src:
    clc_data = src.read(1)
    # The raster transform is necessary for accurate plotting
    transform = src.transform 
    clc_crs = src.crs

# --- 1. Create a Mask for Water Features ---
# Isolate pixels that have the exact water resistance value (800)
water_mask = (clc_data == WATER_RESISTANCE_VALUE)

# Create a new array: where the mask is TRUE, keep the water value (800); 
# otherwise, set to NaN for transparency in the plot
water_only_array = np.where(water_mask, WATER_RESISTANCE_VALUE, np.nan) 
 
# --- 2. Plot the Water Features ---

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Define a simple colormap for just the water (blue)
cmap = mcolors.ListedColormap(['#007FFF']) # A specific shade of blue
bounds = [WATER_RESISTANCE_VALUE, WATER_RESISTANCE_VALUE + 1]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# Plot the AOI boundary for geographic context
aoi_gdf.to_crs(clc_crs).plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1, linestyle='--', label='AOI Boundary')

# Plot the masked water data
rioshow(
    water_only_array,
    transform=transform,
    ax=ax,
    cmap=cmap,
    norm=norm,
    title=f"CLC Fallback Water Features Only (Resistance {WATER_RESISTANCE_VALUE})",
    interpolation='none'
)

# Annotate the plot
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")

# Create a custom legend handle
blue_patch = mpatches.Patch(color='#007FFF', label=f'Water Courses/Bodies (Res={WATER_RESISTANCE_VALUE})')
ax.legend(handles=[blue_patch], loc='lower right')

plt.tight_layout()
